In [1]:

# data processing packages
import numpy as np   
import pandas as pd 

from scipy import stats # look at scipy
from scipy import linalg
from scipy import *
 
%matplotlib inline    
import matplotlib as mplt
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import IPython
from IPython.display import display

import matplotlib
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
# matplotlib.style.use('ggplot')
import itertools

import random

# machine leanring packages

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


from keras.models import Sequential 
from keras.layers.core import Dense, Activation, Dropout, Reshape, Flatten
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.optimizers import *
from keras.callbacks import *
# RMSprop, Adadelta
from keras.regularizers import *
from keras.initializers import *
from keras.activations import *

from utils_keras import *
from utils_dataPrepro import *





Couldn't import dot_parser, loading of dot files will not be possible.


Using Theano backend.


In [4]:
# Notes:

# ---Ini:            
#     orthogonal: https://smerity.com/articles/2016/orthogonal_init.html
#     identity, Xavier, He's

# ---Activation:     
#     tan, sigmod by default

# ---Regularization: 
#     dropout on non-recurrent connections 
#     batch normalization

# unstable or not decreasing training loss: lr, representability, e.g., number of neursons, layers, etc..
# watch out for the amount difference between regularization and loss  
# large minibatch -> large lr
# large network -> large lr


# learning speed, repren, regular

# TO DO:
    
# more component
# muliti-input-muliti-output method
# log transformation
# integrate trend 
# attention for peridoic time series

# multistep ahead:  multi-output rnn
#                   seq2seq rnn 

# Stat-space model: 


In [3]:
# stock data

files_list=["../dataset/dataset_ts/stock_xtrain.dat", \
            "../dataset/dataset_ts/stock_xtest.dat",\
            "../dataset/dataset_ts/stock_ytrain.dat", \
            "../dataset/dataset_ts/stock_ytest.dat"]

xtrain_df = pd.DataFrame( np.load(files_list[0]) )
xtest_df  = pd.DataFrame( np.load(files_list[1]) )
ytrain_df = pd.DataFrame( np.load(files_list[2]) )
ytest_df  = pd.DataFrame( np.load(files_list[3]) )


# files_list=["../dataset/dataset_ts/power_xtrain.csv", \
#             "../dataset/dataset_ts/power_xtest.csv",\
#             "../dataset/dataset_ts/power_ytrain.csv", \
#             "../dataset/dataset_ts/power_ytest.csv"]

# xtrain_df = pd.read_csv( files_list[0] ,sep=',', header=None)
# xtest_df  = pd.read_csv( files_list[1] ,sep=',', header=None)
# ytrain_df = pd.read_csv( files_list[2] ,sep=',', header=None)
# ytest_df  = pd.read_csv( files_list[3] ,sep=',', header=None)

xtrain, ytrain, xtest, ytest = prepare_train_test_data( 200, False, xtrain_df, xtest_df, ytrain_df, ytest_df)
    
print np.shape(xtrain), np.shape(ytrain), np.shape(xtest), np.shape(ytest)



(3040, 200) (3040, 1) (760, 200) (760, 1)


In [6]:
# plain discriminative 

# network set-up
in_out_neurons = 1
hidden_neurons = 256
win_size = 200

batch_size = 256

disc_xtrain = np.reshape( xtrain, [-1, win_size, in_out_neurons] )
disc_xtest  = np.reshape( xtest,  [-1, win_size, in_out_neurons] )

disc_ytrain = ytrain
disc_ytest  = ytest

print np.shape(disc_xtrain),np.shape(disc_ytrain),np.shape(disc_xtest),\
np.shape(disc_ytest)

# optimizer
sgd  = SGD(lr = 0.01, momentum = 0.9, nesterov = True)
rms  = RMSprop(lr = 0.05,  rho = 0.9, epsilon  = 1e-08, decay = 0.0)


adam = Adam(lr = 0.005, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, decay = 0.0)


model = Sequential()
model.add( LSTM(hidden_neurons, input_dim = in_out_neurons, return_sequences = False, \
                input_shape = [batch_size, win_size, in_out_neurons ], \
#               input_length = win_size, \
#               activation ='tanh',\
#               dropout = 0.1,\
#               kernel_regularizer = l2(0.2), 
#               recurrent_regularizer = l2(0.1),\
#               !change!
                kernel_initializer    = glorot_normal(), \
                recurrent_initializer = glorot_normal() ))
# change: activiation

# model.add( LSTM(hidden_neurons, return_sequences = False, \
# #               input_shape = [batch_size, win_size, in_out_neurons ], \
# #               input_length = win_size, \
#               activation ='tanh',\
# #               dropout = 0.1,\
# #               kernel_regularizer = l2(0.2), 
# #               recurrent_regularizer = l2(0.1),\
# #               !change!
#                 kernel_initializer    = glorot_normal(), \
#                 recurrent_initializer = glorot_normal() ))


# identity ini
# he's ini:   he_normal()
# xavier ini: glorot_normal()
# orthogonal: Orthogonal()


# model.add(Dropout(0.5))
model.add(Dense(128,\
#                   kernel_regularizer = l2(0.01), \
                    kernel_initializer = he_normal()))
model.add(Activation("relu"))
model.add(Dense(64, \
#                   kernel_regularizer = l2(0.01), \
                    kernel_initializer = he_normal()))
model.add(Activation("relu"))
model.add(Dense(32, \
#                   kernel_regularizer = l2(0.01), \
                    kernel_initializer = he_normal()))
model.add(Activation("relu"))
model.add(Dense(1,  \
#                   kernel_regularizer = l2(0.01), \
                    kernel_initializer = he_normal()))
model.add(Activation("linear"))


# training
model.compile(loss = "mean_squared_error", optimizer = adam)


model.fit( disc_xtrain, disc_ytrain, shuffle = True, \
           callbacks = [ \
           TestCallback( (disc_xtest, disc_ytest), (disc_xtrain, disc_ytrain) ) ], \
           batch_size = batch_size, epochs = 500)


(3040, 200, 1) (3040, 1) (760, 200, 1) (760, 1)
Epoch 1/500
2816/3040 [==========================>...] - ETA: 11s - loss: 1692548.9432
Testing loss: 1274001.30658, test_err:1128.7166653, train_err:1134.52304366 

3040/3040 [==============================] - 238s - loss: 1670629.6632    
Epoch 2/500
2816/3040 [==========================>...] - ETA: 10s - loss: 577618.1882
Testing loss: 170779.337007, test_err:413.25455469, train_err:408.130061653 

3040/3040 [==============================] - 216s - loss: 549117.8102    
Epoch 3/500
2816/3040 [==========================>...] - ETA: 13s - loss: 93828.6282
Testing loss: 56495.6588405, test_err:237.688153522, train_err:246.989565854 

3040/3040 [==============================] - 256s - loss: 91936.7611    
Epoch 4/500
2816/3040 [==========================>...] - ETA: 11s - loss: 56690.6946
Testing loss: 51846.9339227, test_err:227.699219889, train_err:224.548743943 

3040/3040 [==============================] - 226s - loss: 55688.9144    


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:34: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:34: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(256, return_sequences=False, recurrent_initializer=<keras.ini..., kernel_initializer=<keras.ini..., input_shape=(None, 1))`
ERROR: 

Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/IPython/core/ultratb.py", line 1118, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python2.7/dist-packages/IPython/core/ultratb.py", line 300, in wrapped
    return f(*args, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/IPython/core/ultratb.py", line 345, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/usr/lib/python2.7/inspect.py", line 1044, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))


Internal Python error in the inspect module.
Below is the traceback from this internal error.



  File "/usr/lib/python2.7/inspect.py", line 1004, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/usr/lib/python2.7/inspect.py", line 454, in getsourcefile
    if hasattr(getmodule(object, filename), '__loader__'):
  File "/usr/lib/python2.7/inspect.py", line 490, in getmodule
    for modname, module in sys.modules.items():
KeyboardInterrupt



Unfortunately, your original traceback can not be constructed.



IndexError: string index out of range

In [12]:
# Generative model

# set-up
in_out_neurons = 1
hidden_neurons = 256
win_size = 200    
    
# prepare data
ytrain = expand_y( xtrain_df.as_matrix(), ytrain_df.as_matrix() )
ytest  = expand_y( xtest_df.as_matrix(),  ytest_df.as_matrix()  )

gen_xtrain = np.reshape( xtrain, [-1, win_size, 1] )
gen_ytrain = np.reshape( ytrain, [-1, win_size, 1] )

gen_xtest  = np.reshape( xtest, [-1, win_size, 1] )
gen_ytest  = np.reshape( ytest, [-1, win_size, 1] )

print np.shape(gen_xtrain), np.shape(gen_ytrain), np.shape(gen_xtest), np.shape(gen_ytest)


# optimizer 
adam  = Adam(lr = 0.002, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, decay = 0.0)

model = Sequential()
model.add( LSTM(hidden_neurons, input_dim = in_out_neurons, return_sequences = True, \
                input_length = win_size,\
                activation   = 'tanh',\
#                dropout = 0.1,\
#                kernel_regularizer = l2(0.15), \
#                recurrent_regularizer = l2(0.15), \
                kernel_initializer    = glorot_normal(), \
                recurrent_initializer = glorot_normal() ) )


# identity ini
# he's ini:   he_normal()
# xavier ini: glorot_normal()
# orthogonal: Orthogonal() 


# model.add(Dropout(0.1))
model.add(TimeDistributed(Dense(128, activation = 'relu',\
#                                     kernel_regularizer = l2(0.1), \
                                    kernel_initializer = he_normal() ) ))

model.add(TimeDistributed(Dense(64, activation = 'relu',\
#                                     kernel_regularizer = l2(0.01), \
                                    kernel_initializer = he_normal() ) ))

model.add(TimeDistributed(Dense(1,  activation = 'linear',\
#                                     kernel_regularizer = l2(0.01), \
                                    kernel_initializer = he_normal() ) ))


# training
model.compile( loss = "mean_squared_error", optimizer = adam )


model.fit( gen_xtrain, gen_ytrain, shuffle=True,  \
           callbacks = [ TestCallback_Generative( \
                         (gen_xtest, gen_ytest), (gen_xtrain, gen_ytrain) ) ], \
                         batch_size = 128, epochs = 500 )

In [ ]:
# discriminative with non-overlapping local data

#  network set-up
local_size = 5
in_out_neurons = local_size
hidden_neurons = 512
win_size = 200/5

# # validation on each epoch 
# class TestCallback_insight(Callback):
#     def __init__(self, test_data):
#         self.test_data = test_data

#     def on_epoch_end(self, epoch, logs={}):
#         x, y = self.test_data
#         loss, mse = self.model.evaluate(x, y, verbose=0)
        
#         py = self.model.predict(x, verbose=0)
#         cnt = len(y)
#         err = [ (y[i][0]-py[i][0])**2 for i in range(cnt)]
        
#         print('\nTesting loss: {}, acc: {}, mean: {} \n'.format(\
#                loss, sqrt(mse), mean(err) ))

def expand_x_local_non_overlapping( local_size, list_data):
    
    cnt = len(list_data)
    steps = len(list_data[0])
    tmp_dta = []
    
    for i in range(cnt):
        tmp_dta.append([])
        for j in range( local_size-1, steps, local_size ):
            tmp_dta[-1].append( list_data[i][ j-local_size+1:j+1 ] )
    
    return tmp_dta


disc_xtrain = np.array( expand_x_local_non_overlapping( local_size, list(xtrain) ) )
disc_xtest  = np.array( expand_x_local_non_overlapping( local_size, list(xtest) ) )

disc_ytrain = ytrain
disc_ytest  = ytest

print np.shape(disc_xtrain), np.shape(disc_ytrain), np.shape(disc_xtest), \
np.shape(disc_ytest)

# optimizer 
adam = Adam(lr = 0.007, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, decay = 0.0)

model = Sequential()
model.add(LSTM(hidden_neurons, input_dim = in_out_neurons, return_sequences = False, \
               input_length = win_size, \
               activation = 'tanh', \
#                dropout = 0.1,\
#                kernel_regularizer = l2(0.1),      recurrent_regularizer = l2(0.1), \
               kernel_initializer    = glorot_normal(), \
               recurrent_initializer = glorot_normal() ))
# identity ini
# he's ini:   he_normal()
# xavier ini: glorot_normal()
# orthogonal: Orthogonal() 

model.add(Dropout(0.1))

# kernel_regularizer = l2(0.1), \
model.add(Dense(128,\
                    kernel_initializer = he_normal()))
model.add(Activation("relu"))
model.add(Dense(64, \
                    kernel_initializer = he_normal()))
model.add(Activation("relu"))
model.add(Dense(32, \
                    kernel_initializer = he_normal()))
model.add(Activation("relu"))
model.add(Dense(1,\
                    kernel_initializer = he_normal()))
model.add(Activation("linear"))


# training 
model.compile(loss="mean_squared_error", optimizer = adam)


model.fit( disc_xtrain, disc_ytrain, shuffle=True, \
           callbacks = [ TestCallback( (disc_xtest, disc_ytest), (disc_xtrain, disc_ytrain) ) ], \
           batch_size = 256, epochs = 500)

In [ ]:
# from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model
from keras.layers import *
from keras.layers.merge import *
from keras import backend as K


in_out_neurons = 1
hidden_neurons = 256
win_size = 200

batch_size = 256


input_raw  = Input(shape = ( win_size, 1 ),    dtype='float32', name='input_raw')
input_diff = Input(shape = ( win_size -1, 1 ), dtype='float32', name='input_diff')
input_last = Input(shape = ( 1, ), dtype='float32', name='input_last')

hidden_raw  = LSTM(32)( input_raw )
output_raw  = Dense(1, activation='relu', name='aux_output')(hidden_raw)

hidden_diff = LSTM(32)( input_diff )
output_diff = Dense(1, activation='relu', name='aux_output')(hidden_diff)
output_diff = Add( [output_diff, input_last] )

attention_diff = Add( [output_diff, -1*output_raw] )
attention_prob = Dense(1, activation='sigmoid')( attention_diff )
attention_diff = merge([output_diff, attention_prob], name='attention_diff', mode='mul')


tmp_batch_sizee = int(input_raw.shape[0])
ones = K.ones((tmp_batch_sizee,1))

attention_prob = Add( [one, -1*attention_prob] )
attention_raw  = merge([output_raw, attention_prob], name='attention_raw', mode='mul')

# x = keras.layers.concatenate([lstm_out, auxiliary_input])
# x = Dense(64, activation='relu')(x)

output_main = Add( [attention_diff, attention_raw] )

# And finally we add the main logistic regression layer
# main_output = Dense(1, activation='sigmoid', name='main_output')(x)


model = Model(inputs=[ input_raw, input_diff, input_last], outputs=[ output_main ])

model.compile(optimizer = adam, loss='mean_squared_error',
#               loss_weights=[1., 0.2]\
             )

model.fit([headline_data, additional_data], [labels], epochs = 500, batch_size = batch_size )



# def attention_3d_block(inputs):
#     # inputs.shape = (batch_size, time_steps, input_dim)
#     input_dim = int(inputs.shape[2])
#     a = Permute((2, 1))(inputs)
#     a = Reshape((input_dim, TIME_STEPS))(a)
#     a = Dense(TIME_STEPS, activation='softmax')(a)
#     if SINGLE_ATTENTION_VECTOR:
#         a = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(a)
#         a = RepeatVector(input_dim)(a)
#     a_probs = Permute((2, 1), name='attention_vec')(a)
#     output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
#     return output_attention_mul


# def model_attention_applied_after_lstm():
#     inputs = Input(shape=(TIME_STEPS, INPUT_DIM,))
#     lstm_units = 32
#     lstm_out = LSTM(lstm_units, return_sequences=True)(inputs)
#     attention_mul = attention_3d_block(lstm_out)
#     attention_mul = Flatten()(attention_mul)
#     output = Dense(1, activation='sigmoid')(attention_mul)
#     model = Model(input=[inputs], output=output)
#     return model


